# A basic lsdtopotools analysis

Written by Simon M. Mudd at the University of Edinburgh. Updated 02/09/2021

We assume you are in our docker container. 

This includes `lsdviztools` and `lsdparamselector`.

If you are not in our docker container you will need to pip install these packages. 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import lsdttparamselector as ps

## Part 1: grab data

First, we need to grab some data. We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

You can tell this what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from goole earth by right clickjing on the map and selecting "what's here". In this example I use a place near Santa Barbara, California. 

In [ ]:
SB_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -120.464655, longitude_E = -120.254214, latitude_S = 34.440538, latitude_N = 34.610770,prefix = "lg_conception")
SB_DEM.print_parameters()

Okay, we haven't actually got any data. We need to do that by using the `download_pythonic` option. When it is finished it will say `finished downloading`. 

In [ ]:
SB_DEM.download_pythonic()

Okay, now we need to do two things. First, we need to convert to UTM. Then we need to convert to ENVI bil format. 

In [ ]:
SB_DEM.to_UTM_pythonic()
RasterFile = "lg_conception_SRTM30_UTM.tif"
DataDirectory = "./"
gio.convert2bil(DataDirectory, RasterFile,minimum_elevation=0)

Okay, we now have an ENVI bil DEM. We are ready to start doing topographic analysis!

# Part 2: Topographic analysis

Now, that we have the data, we need to use an `lsdttparamselector` object to select parameters. The current default is to initiate a selector using the command line tool `lsdtt-basic-metrics`.

In [ ]:
param = ps.lsdttdm()

Okay, now lets initiate the widget and then call it to enter the parameters we want.

In [ ]:
tab_nest = param.make_widget()
tab_nest

Now that we have done that, we grab the parameter values. 

In [ ]:
parameter_dict = param.read_widgets()
print(parameter_dict)

Okay, now we initiate an `lsdtt_driver` object, which handles the call to `lsdtt-basic-metrics`.

In [ ]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "lg_conception_SRTM30_UTM",write_prefix= "lg_conception_SRTM30_UTM",parameter_dictionary=parameter_dict)
lsdtt_drive.print_parameters()

Now run the tool!

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

Super. We will now have a bunch of rasters and csv data ready for visualisation. 

# Part 3: plot some data

Okay, now that we have some rasters, we can use lsdviztools to plot some data. First let's do a hillshade plot. We'll use a `gist_earth` colourmap. This makes an image file (that you can use later). 

In [ ]:
DataDirectory = "./"
Base_file = "lg_conception_SRTM30_UTM"
    
img_name = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth")

We render the code with a markdown call (it looks like this `<img src="lg_conception_SRTM30_UTM_hillshade.png" width=600>`)

In [ ]:
img_name

We can display this image

In [ ]:
from IPython.display import Image
Image(filename=img_name,width=800) 

Now lets print the slope map

In [ ]:
Drape_prefix = "lg_conception_SRTM30_UTM_SLOPE"
img_name2 = lsdmw.SimpleDrape(DataDirectory,Base_file, Drape_prefix, cmap = "gnuplot", cbar_loc = "right", cbar_label = "Gradient (m/m)")

In [ ]:
img_name2

<img src="lg_conception_SRTM30_UTM_drape.png" width=600>

In [ ]:
from IPython.display import Image
Image(filename=img_name2,width=800) 